In [25]:
import pandas as pd
import numpy as np

from collections import Counter
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import load_model

from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
df_train = pd.read_csv("korean_unsmile_dataset-main/unsmile_train_new.csv")
df_test = pd.read_csv("korean_unsmile_dataset-main/unsmile_valid_new.csv")

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22532 entries, 0 to 22531
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   문장      22532 non-null  object
 1   target  22532 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 352.2+ KB


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5604 entries, 0 to 5603
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   문장      5604 non-null   object
 1   target  5604 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 87.7+ KB


In [7]:
df_train

,문장,target
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,1
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1
...,...,...
22527,마음의 미동을주고싶은데마음먹은듯한데 어떻게서든 돌리고싶습니다도와주세요,0
22528,그엄마가 그엄마는 아니겠지,0
22529,연락안한지 3일됬는데..너무 __ 즉흥적으로 질러버린거같아요..,0
22530,엄마아빠가 이해해주실까요?,0


In [8]:
df_test

,문장,target
0,ㅇㄱㄹㅇ 진짜 죽어도 상관없다는 마인드로 싸웠더니 지금 서열 상타취노 식칼들고 니가...,1
1,여자들은 취미가 애낳는건가.. 취미를 좀 가져라,1
2,개슬람녀 다 필요없고 니 엄마만 있으면 된다,1
3,조팔ㅋㅋ 남한 길거리 돌아다니면 한국남자때문에 눈재기하는데 그걸 내 폰에 굳이 담아...,1
4,바지 내리다 한남들 와꾸 보고 올려뿟노,1
...,...,...
5599,멀 어떻게 해야되요??,0
5600,프로필사진에는 아랍인처럼 안생겼던데..,0
5601,난 뭔가 좀 불안한데... 이렇게 잘하다가 한순간 못하면 지금까지 넣은 골같은건 상...,0
5602,벌써 4시를 향해가고 있음...,0


In [12]:
okt = Okt()
malist = okt.pos("아버지 가방에 들어가신다. ㅅㅂ", norm=True, stem=True)
malist2 = okt.pos("아버지 가방에 들어가신다. ㅅㅂ ㅄ ㅂㅅ", norm=False, stem=True)
print(malist)
print(malist2)

# 'KoreanParticle': '(ex: ㅋㅋ)',

[('아버지', 'Noun'), ('가방', 'Noun'), ('에', 'Josa'), ('들어가다', 'Verb'), ('.', 'Punctuation'), ('ㅅㅂ', 'KoreanParticle')]
[('아버지', 'Noun'), ('가방', 'Noun'), ('에', 'Josa'), ('들어가다', 'Verb'), ('.', 'Punctuation'), ('ㅅㅂ', 'KoreanParticle'), ('ㅄ', 'KoreanParticle'), ('ㅂㅅ', 'KoreanParticle')]


In [48]:
def make_corpus(sentence):
    okt = Okt()
    raw_pos_tagged = okt.pos(sentence, norm=True, stem=True)
    word_cleaned = []

    for word in raw_pos_tagged:
        if word[1] in ['Noun', 'Adjective', 'Verb']:
            word_cleaned.append(word[0])
    return ' '.join(word_cleaned)

In [49]:
df_train.head()

,문장,target,tokenized
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,이다 시간 쉬다 그렇다 아니다
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,1,아동 성범죄 페도버 기록 바 끊어지다 영원하다 고통 받다 무슬림 친 떨어지다 출산 위험
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,루나 솔로 앨범 나오다 때 머모 기운 있다 ㅇㅇ 진짜 띵곡임 꼭 들다 보다
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,홍팍 어버이연합 보내다 뭐 뎃글 있다 거 어버이연합 신고 하다 쪽 고소 가능하다
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,놓다 왜 여기 댓 여자 김치 녀 먼저 불리다 여자 더 심하다 그렇다 개 싸우다 내 ...


In [50]:
df_train['tokenized'] = df_train['문장'].map(make_corpus)
df_test['tokenized'] = df_test['문장'].map(make_corpus)

In [51]:
df_train.head()

,문장,target,tokenized
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,이다 시간 쉬다 그렇다 아니다
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,1,아동 성범죄 페도버 기록 바 끊어지다 영원하다 고통 받다 무슬림 친 떨어지다 출산 위험
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,루나 솔로 앨범 나오다 때 머모 기운 있다 진짜 띵곡임 꼭 들다 보다
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,홍팍 어버이연합 보내다 뭐 뎃글 있다 거 어버이연합 신고 하다 쪽 고소 가능하다
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,놓다 왜 여기 댓 여자 김치 녀 먼저 불리다 여자 더 심하다 그렇다 개 싸우다 내 ...


In [52]:
df_test.head()

,문장,target,tokenized
0,ㅇㄱㄹㅇ 진짜 죽어도 상관없다는 마인드로 싸웠더니 지금 서열 상타취노 식칼들고 니가...,1,진짜 죽다 상관없다 마인드 싸우다 지금 서열 상타취노 식칼 니 나 찌르다 내 너 찌...
1,여자들은 취미가 애낳는건가.. 취미를 좀 가져라,1,여자 취미 애 낳다 건가 취미 좀 가다
2,개슬람녀 다 필요없고 니 엄마만 있으면 된다,1,개슬람녀 필요없다 니 엄마 있다 되다
3,조팔ㅋㅋ 남한 길거리 돌아다니면 한국남자때문에 눈재기하는데 그걸 내 폰에 굳이 담아...,1,조팔 남한 길거리 돌아다니다 한국 남자 때문 눈 재다 내 폰 굳이 담다 야하다 조팔
4,바지 내리다 한남들 와꾸 보고 올려뿟노,1,바지 리다 한남 오다 꾸다 보고 올려뿟노


In [53]:
tfidf_vect = TfidfVectorizer(max_features=100)
tfidf_vect.fit(df_train.tokenized)
df_train_tfidf = tfidf_vect.transform(df_train.tokenized)
df_test_tfidf = tfidf_vect.transform(df_test.tokenized)

In [22]:
type(df_train_tfidf)

scipy.sparse.csr.csr_matrix

In [23]:
df_test_tfidf.shape

(5604, 100)

In [54]:
df_train_tfidf = df_train_tfidf.toarray()
df_test_tfidf = df_test_tfidf.toarray()

In [55]:
X_train = df_train_tfidf
X_test = df_test_tfidf
y_train = df_train['target']
y_test = df_test['target']

In [56]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (22532, 100)
X_test shape:  (5604, 100)
y_train shape:  (22532,)
y_test shape:  (5604,)


In [57]:
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(6, embedding_dim))
# model.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM을 사용
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=5, callbacks=[es, mc], batch_size=256, validation_split=0.2)

Epoch 1/5
71/71 [==============================] - 48s 630ms/step - loss: 0.6637 - acc: 0.6204 - val_loss: 0.9337 - val_acc: 0.0000e+00

Epoch 00001: val_acc improved from -inf to 0.00000, saving model to best_model.h5
Epoch 2/5
71/71 [==============================] - 46s 651ms/step - loss: 0.6596 - acc: 0.6292 - val_loss: 0.9144 - val_acc: 0.0963

Epoch 00002: val_acc improved from 0.00000 to 0.09629, saving model to best_model.h5
Epoch 3/5
71/71 [==============================] - 51s 723ms/step - loss: 0.6594 - acc: 0.6293 - val_loss: 1.0229 - val_acc: 0.0996

Epoch 00003: val_acc improved from 0.09629 to 0.09962, saving model to best_model.h5
Epoch 4/5
71/71 [==============================] - 50s 701ms/step - loss: 0.6524 - acc: 0.6353 - val_loss: 0.7902 - val_acc: 0.1498

Epoch 00004: val_acc improved from 0.09962 to 0.14977, saving model to best_model.h5
Epoch 5/5
71/71 [==============================] - 51s 722ms/step - loss: 0.6514 - acc: 0.6371 - val_loss: 1.0218 - val_acc: 0.

In [33]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

176/176 [==============================] - 6s 32ms/step - loss: 0.7034 - acc: 0.5289

 테스트 정확도: 0.5289


In [62]:
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(6, embedding_dim))
model.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM을 사용
# model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=5, callbacks=[es, mc], batch_size=256, validation_split=0.2)

Epoch 1/5
71/71 [==============================] - 130s 2s/step - loss: 0.6666 - acc: 0.6212 - val_loss: 0.9908 - val_acc: 0.0000e+00

Epoch 00001: val_acc improved from -inf to 0.00000, saving model to best_model.h5
Epoch 2/5
71/71 [==============================] - 154s 2s/step - loss: 0.6613 - acc: 0.6250 - val_loss: 1.0123 - val_acc: 0.0000e+00

Epoch 00002: val_acc did not improve from 0.00000
Epoch 3/5
71/71 [==============================] - 155s 2s/step - loss: 0.7716 - acc: 0.5856 - val_loss: 0.6109 - val_acc: 0.9470

Epoch 00003: val_acc improved from 0.00000 to 0.94697, saving model to best_model.h5
Epoch 4/5
71/71 [==============================] - 147s 2s/step - loss: 0.6616 - acc: 0.6170 - val_loss: 0.9230 - val_acc: 0.0941

Epoch 00004: val_acc did not improve from 0.94697
Epoch 5/5
71/71 [==============================] - 139s 2s/step - loss: 0.6700 - acc: 0.6190 - val_loss: 0.9033 - val_acc: 0.0353

Epoch 00005: val_acc did not improve from 0.94697


In [63]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

176/176 [==============================] - 16s 79ms/step - loss: 0.7043 - acc: 0.4823

 테스트 정확도: 0.4823
